## 数据集
### 原始数据

In [32]:
import tensorflow as tf

import pandas as pd

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

# 120 * 5
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
# 30 * 5
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [2]:
## check case
train

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0
...,...,...,...,...,...
115,5.5,2.6,4.4,1.2,1
116,5.7,3.0,4.2,1.2,1
117,4.4,2.9,1.4,0.2,0
118,4.8,3.0,1.4,0.1,0


### 构建数据集 

In [3]:
# 训练集标签：120 * 1
train_y = train.pop('Species')
# 测试集标签：30 * 1
test_y = test.pop('Species')
# 训练样本 train: 120 * 4 , 测试样本 test: 30 * 4

In [16]:
## check case
# train_y
# train
# train.keys()
dict(train)['SepalLength']

0      6.4
1      5.0
2      4.9
3      4.9
4      5.7
      ... 
115    5.5
116    5.7
117    4.4
118    4.8
119    5.5
Name: SepalLength, Length: 120, dtype: float64

## Estimator 使用
### 创建输入函数

In [6]:
# 输入函数
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # 将输入转换为数据集。
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # 如果在训练模式下混淆并重复数据。
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)


# 评估函数

In [8]:
# 定义特征列
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

### 分类器实例化

In [10]:
# 默认 
## 构建一个拥有两个隐层，隐藏节点分别为 30 和 10 的深度神经网络。
# classifier = tf.estimator.DNNClassifier(
#     feature_columns=my_feature_columns,
#     # 隐层所含结点数量分别为 30 和 10.
#     hidden_units=[30, 10],
#     # 模型必须从三个类别中做出选择。
#     n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpklite7bd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1e3d7d84a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
# 自定义 checkpoints
## Checkpoints 设置
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 20 * 60,
    keep_checkpoint_max = 10
)

## 分类器
# 构建一个拥有两个隐层，隐藏节点分别为 30 和 10 的深度神经网络。
classifier_diy = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # 隐层所含结点数量分别为 30 和 10.
    hidden_units=[30, 10],
    # 模型必须从三个类别中做出选择。
    n_classes=3,
    model_dir = 'models/iris',
    config=my_checkpointing_config
)


INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1200, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1e3d7eb390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### 训练模型

In [23]:
# 训练模型。
classifier_diy.train(
    # 输入函数
    input_fn=lambda: input_fn(train, train_y, training=True),
    # 训练 steps 步后停止训练
    steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 15000 into models/iris/model.ckpt.
INFO:tensorflow:loss = 6.8813686, step = 15001
INFO:tensorflow:global_step/sec: 750.15
INFO:tensorflow:loss = 5.848205, step = 15101 (0.134 sec)
INFO:tensorflow:global_step/sec: 968.011
INFO:tensorflow:loss = 5.6258526, step = 15201 (0.103 sec)
INFO:tensorflow:global_step/sec: 964.656
INFO:tensorflow:loss = 5.4370074, step = 15301 (0.104 sec)
INFO:tensorflow:global_step/sec: 976.053
INFO:tensorflow:loss = 6.0909038, step = 15401 (0.102 sec)
INFO:tensorflow:global_step/sec: 957.757
INFO:tensorflow:loss = 4.659221, step = 15501 (0.104 sec)
INFO:tensorflow:global_step/sec: 952.93
INFO:tensorflow:loss = 5.4505396, 

### 评估模型

In [22]:
# 模型评估
eval_result = classifier_diy.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
eval_result

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-09-13T11:57:24Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2023-09-13-11:57:24
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.96666664, average_loss = 0.18470596, global_step = 15000, loss = 5.5411787
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: models/iris/model.ckpt-15000

Test set accuracy: 0.967



{'accuracy': 0.96666664,
 'average_loss': 0.18470596,
 'loss': 5.5411787,
 'global_step': 15000}

In [24]:
# 模型评估
eval_result = classifier_diy.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
eval_result

## 使用 tensorboard 来查看训练情况  http://127.0.0.1:6006
### tensorboard --logdir models/iris --port=6006

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-09-13T11:57:52Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2023-09-13-11:57:53
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.96666664, average_loss = 0.21294494, global_step = 20000, loss = 6.388348
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: models/iris/model.ckpt-20000

Test set accuracy: 0.967



{'accuracy': 0.96666664,
 'average_loss': 0.21294494,
 'loss': 6.388348,
 'global_step': 20000}

### 模型预测

In [33]:
# 由模型生成预测
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # 将输入转换为无标签数据集。
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier_diy.predict(
    input_fn=lambda: input_fn(predict_x))


for pred_dict, expec in zip(predictions, expected):
    print("pred_dict: {}".format(pred_dict))
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
pred_dict: {'logits': array([ 51.02154 ,  43.691315, -67.36501 ], dtype=float32), 'probabilities': array([9.9934500e-01, 6.5499556e-04, 0.0000000e+00], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" (99.9%), expected "Setosa"
pred_dict: {'logits': array([ -4.486701,  10.502182, -12.314878], dtype=float32), 'probabilities': array([3.0932188e-07, 9.9999964e-01, 1.2321863e-10], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction i

### 模型导出

In [47]:
saved_model_dir = "saved_models/iris"


feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns)
print(feature_spec)
#feature_spec = {'x': tf.FixedLenFeature([4],tf.float32)}

# export_path = classifier_diy.export_saved_model(saved_model_dir,
#     tf.estimator.export.build_raw_serving_input_receiver_fn(features=feature_spec))

# export_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feature_spec)
# export_input_fn = tf.contrib.learn.utils.build_parsing_serving_input_fn(feature_spec)
# classifier_diy.export_savedmodel(saved_model_dir, export_input_fn)

# 正常导出
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
classifier_diy.export_savedmodel(saved_model_dir, export_input_fn)
    
    
# feature_spec = {'x': tf.FixedLenFeature([4],tf.float32)}

# def serving_input_receiver_fn():
#   serialized_tf_example = tf.placeholder(dtype=tf.string,
#                                          shape=[None],
#                                          name='input_tensors')
#   receiver_tensors = {'inputs': serialized_tf_example}
#   features = tf.parse_example(serialized_tf_example, feature_spec)
#   return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

# export_path = classifier_diy.export_savedmodel(saved_model_dir, serving_input_receiver_fn)

{'SepalLength': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None), 'SepalWidth': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None), 'PetalLength': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None), 'PetalWidth': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None)}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from models/iris/model.ckpt-20000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_models/iris/temp-b'1695035326'/saved_model.pb


b'saved_models/iris/1695035326'